In [ ]:
# ============================================================
# Instalação de pacotes no Google Colab
# ============================================================
!pip install torch torchvision torchaudio
!pip install networkx matplotlib

In [ ]:
# ============================================================
# Importações
# ============================================================
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random

# Semente para reprodutibilidade (gera os mesmos resultados em diferentes execuções)
np.random.seed(42)

In [ ]:
# ============================================================
# Construção da rede biológica toy
# ============================================================
num_nodes = 120     # número de nós (proteínas simuladas)
num_edges = 2       # cada novo nó conecta-se a 2 já existentes
G = nx.barabasi_albert_graph(num_nodes, num_edges, seed=42)

print(f"Grafo com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas")

# Número de atributos (features) de cada proteína
num_features = 10

# Definição de classes reais (exemplo: núcleo=0, citoplasma=1)
labels = np.random.randint(0, 2, size=num_nodes)

# Geração de features correlacionadas com as classes
features = np.zeros((num_nodes, num_features))
for i in range(num_nodes):
    if labels[i] == 0:
        # proteínas do "núcleo" → valores médios positivos
        features[i] = np.random.normal(loc=0.5, scale=0.3, size=num_features)
    else:
        # proteínas do "citoplasma" → valores médios negativos
        features[i] = np.random.normal(loc=-0.5, scale=0.3, size=num_features)
features = features.astype(np.float32)

In [ ]:
# ============================================================
# Preparação dos dados para a GCN
# ============================================================
A = nx.to_numpy_array(G)   # matriz de adjacência (ligações entre proteínas)
I = np.eye(num_nodes)      # matriz identidade (self-loops)
A_hat = A + I              # adiciona self-loops → cada nó se conecta a si mesmo

# Normalização simétrica: D^(-1/2) * A_hat * D^(-1/2)
D_hat = np.diag(np.sum(A_hat, axis=1))          # grau dos nós
D_hat_inv_sqrt = np.linalg.inv(np.sqrt(D_hat))  # inverso da raiz quadrada
A_norm = D_hat_inv_sqrt @ A_hat @ D_hat_inv_sqrt  # matriz normalizada

# Conversão para tensores do PyTorch
X = torch.tensor(features)                             # features
Y = torch.tensor(labels, dtype=torch.long)             # rótulos
A_norm = torch.tensor(A_norm, dtype=torch.float32)     # matriz adjacência normalizada

In [ ]:
# ============================================================
# Definição da GCN
# ============================================================
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)  # transformação linear

    def forward(self, X, A_norm):
        # Multiplica a matriz normalizada pelas features (propagação de vizinhos)
        # e aplica transformação linear
        return self.linear(A_norm @ X)


class GCN(nn.Module):
    def __init__(self, in_features, hidden_size, num_classes, dropout=0.5):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_features, hidden_size)     # primeira camada GCN
        self.gcn2 = GCNLayer(hidden_size, num_classes)     # segunda camada GCN
        self.dropout = nn.Dropout(dropout)                 # regularização (evita overfitting)

    def forward(self, X, A_norm):
        h = F.relu(self.gcn1(X, A_norm))   # aplica ReLU após primeira camada
        h = self.dropout(h)                # aplica dropout
        out = self.gcn2(h, A_norm)         # logits finais (sem softmax)
        return out

In [ ]:
# ============================================================
# Divisão dos dados (treino, validação, teste)
# ============================================================
num_nodes = X.shape[0]
idx_all = np.arange(num_nodes)
np.random.shuffle(idx_all)  # embaralha nós

# 70% treino, 15% validação, 15% teste
train_end = int(0.70 * num_nodes)
val_end = int(0.85 * num_nodes)
train_idx = idx_all[:train_end]
val_idx = idx_all[train_end:val_end]
test_idx = idx_all[val_end:]


In [ ]:
# ============================================================
# Configuração de treino
# ============================================================
model = GCN(num_features, hidden_size=32, num_classes=2)

lr = 0.01                   # taxa de aprendizado
weight_decay = 5e-4         # regularização L2
max_epochs = 500            # número máximo de épocas
patience = 20               # early stopping (paciência)
monitor = 'val_loss'        # critério para parar (pode ser val_loss ou val_acc)

# Otimizador e função de perda
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

# Controle do early stopping
best_state = None
best_metric = float('inf') if monitor == 'val_loss' else -float('inf')
epochs_no_improve = 0

# Histórico para plots
train_losses, val_losses = [], []
train_accs, val_accs = [], []

In [ ]:
# ============================================================
# Função auxiliar de avaliação (loss e acurácia)
# ============================================================
def evaluate_model(model, X, A_norm, idx):
    model.eval()
    with torch.no_grad():
        out = model(X, A_norm)              # logits (N, C)
        loss = criterion(out[idx], Y[idx]).item()
        preds = out[idx].argmax(dim=1)      # classes previstas
        acc = (preds == Y[idx]).float().mean().item()
    return loss, acc, preds

In [ ]:
# ============================================================
# Loop de treino com Early Stopping
# ============================================================
for epoch in range(1, max_epochs + 1):
    # ---------------- Treino ----------------
    model.train()
    optimizer.zero_grad()
    out = model(X, A_norm)                              # forward
    loss = criterion(out[train_idx], Y[train_idx])      # calcula perda
    loss.backward()                                     # backpropagation
    optimizer.step()                                    # atualização dos pesos

    # ---------------- Avaliação ----------------
    train_loss, train_acc, _ = evaluate_model(model, X, A_norm, train_idx)
    val_loss, val_acc, _ = evaluate_model(model, X, A_norm, val_idx)

    # salva histórico
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    # ---------------- Early Stopping ----------------
    current_metric = val_loss if monitor == 'val_loss' else val_acc
    improved = (current_metric < best_metric) if monitor == 'val_loss' else (current_metric > best_metric)

    if improved:
        best_metric = current_metric
        best_state = copy.deepcopy(model.state_dict())  # salva melhor versão
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # imprime a cada 10 épocas
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | "
              f"val_loss: {val_loss:.4f} val_acc: {val_acc:.4f} | no_improve: {epochs_no_improve}")

    # para se não houver melhora após "patience"
    if epochs_no_improve >= patience:
        print(f"Early stopping na epoch {epoch}. Melhor {monitor}: {best_metric:.4f}")
        break

# carrega melhor modelo salvo
if best_state is not None:
    model.load_state_dict(best_state)

In [ ]:
# ============================================================
# Avaliação no conjunto de teste
# ============================================================
test_loss, test_acc, test_preds = evaluate_model(model, X, A_norm, test_idx)
print(f"Teste | loss: {test_loss:.4f} | acc: {test_acc:.4f}")

In [ ]:
# ============================================================
# Plots de aprendizado
# ============================================================
plt.figure(figsize=(12,4))

# Curva de Loss
plt.subplot(1,2,1)
plt.plot(train_losses, label='train_loss')
plt.plot(val_losses, label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss (Treino vs Val)')

# Curva de Acurácia
plt.subplot(1,2,2)
plt.plot(train_accs, label='train_acc')
plt.plot(val_accs, label='val_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy (Treino vs Val)')

plt.show()

In [ ]:
# ---------------- Rede final ----------------
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
nx.draw(G, pos, node_color=Y.numpy(), cmap=plt.cm.Set1, with_labels=False, node_size=50)
plt.title("Classes reais")

plt.subplot(1,2,2)
# predições para todos os nós
model.eval()
with torch.no_grad():
    out_all = model(X, A_norm)
    _, preds_all = torch.max(out_all, dim=1)
nx.draw(G, pos, node_color=preds_all.numpy(), cmap=plt.cm.Set1, with_labels=False, node_size=50)
plt.title("Classes previstas (modelo final)")

plt.show()